In [1]:
import pandas as pd
import datetime
import numpy as np
from private_stuff.nombres import normalizacion_nombres
from private_stuff.nacionalidades import normalizacion_paises
from private_stuff.institucion import normalizacion_universidades
from funciones import limpiar_caracteres_especiales, limpiar_funciones, instrumento_financiamiento, institucion_instrumento, descripcion_codigo_inach, region_institucion, macrozona_region

### Eliminar personas duplicadas
Lo que se hizo fue tomar cada hoja del procien y eliminar las personas duplicadas bajo el criterio que compartan mismo codigo y mismo anio procien.
En caso de que uno de estos dos sea diferente, esa persona se mantiene.

In [ ]:
new_xls = []
lista_repetidos = []
lista_archivos = ['50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60']

for f in lista_archivos:
    if f in ['50', '51', '55', '56', '57']:
        xls_list = pd.read_excel(f"raw-data/eca/ECA {f}.xlsx").to_numpy().tolist()
    else:
        xls_list = pd.read_excel(f"raw-data/eca/ECA {f}.xlsx", sheet_name=['personas'])['personas'].to_numpy().tolist()
    for row in xls_list:
        if pd.isna( row[0] ):
            pass
        ##Cosas que se buscar
        #codigo, titulo, pi, inst_pi, nombre+apellido, inst, nac, genero y funcion
        if f in ['50', '52']:
            codigo, titulo, pi, inst_pi, nombre, apellido, inst, nac, genero, funcion = row
        elif f in ['51']:
            codigo, titulo, pi, inst_pi, nombre, apellido, inst, nac, genero = row
            funcion = 'noinf'
        elif f in ['53']:
            codigo, pi, nombre, apellido, _, genero, inst_pi, titulo = row
            inst = 'noinf'
            nac = 'noinf'
            funcion = 'noinf'
        elif f in ['57']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, _ = row
            inst_pi = 'noinf'
            funcion = 'noinf'
        elif f in ['54']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, _ , inst_pi, _= row
            funcion = 'noinf'
        elif f in ['55', '56']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, funcion, _ = row
            inst_pi = 'noinf'
        elif f in ['58', '59', '60']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, funcion, _, inst_pi = row
        else:
            pass
        
        
        codigo = codigo.split( ' ' )[0].replace( '_', '-' )
        proyecto, tesis, _ = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        anio = 'noinf'
        if len( codigo.split('-') ) != 3:
            pass
        else:
            anio = f"20{ codigo.split('-')[2] }"

        titulo = limpiar_caracteres_especiales( titulo.lower() )

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        nombre = limpiar_caracteres_especiales( nombre.lower() ).replace('-', ' ').replace(',', '').replace('*', '').strip()
        apellido = limpiar_caracteres_especiales( apellido.lower() ).replace('-', ' ').replace(',', '').replace('*', '').strip()
        nombre = f"{ nombre } { apellido }".strip()
        nombre = normalizacion_nombres[nombre]


        inst = limpiar_caracteres_especiales( inst.lower().strip() )
        inst = normalizacion_universidades[ inst ]
        region_inst = region_institucion[inst]
        macro_inst = macrozona_region[region_inst]

        inst_pi = limpiar_caracteres_especiales( inst_pi.lower().strip() )
        inst_pi = normalizacion_universidades[ inst_pi ]
        region_inst_pi = region_institucion[inst_pi]
        macro_inst_pi = macrozona_region[region_inst_pi]

        nac = normalizacion_paises[ nac.strip().upper() ].lower().strip()

        funcion = limpiar_funciones( limpiar_caracteres_especiales( funcion.lower() ) )


        genero = genero.strip().upper()
        if genero == 'MASCULINO':
            genero = 'M'
        elif genero == 'FEMENINO':
            genero = 'F'
        elif genero == 'NOINF':
            genero = genero.lower()

        if [f, codigo, nombre] not in lista_repetidos:
            new_xls.append([f, anio, codigo, instrumento, proyecto, inst_instrumento, tesis, titulo, 
                pi, inst_pi, region_inst_pi, macro_inst_pi, 
                nombre, inst, region_inst, macro_inst, nac, genero, funcion])
            lista_repetidos.append( [f, codigo, nombre] )
    print(f"{f} done")

In [3]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['eca', 'anio', 'codigo', 'instrumento', 'inst-instrumento', 'tipo-proyecto-inach', 'tesis', 
    'titulo', 'pi', 'inst-pi', 'region-inst-pi', 'macro-inst-pi', 
    'nombre', 'inst', 'region-inst', 'macro-inst', 'nac', 'genero', 'funcion'])
full_dataframe.to_excel(f"eca/ECA-just-personas.xlsx")

### Codigos por anio
Lo que se hace es dejar los codigos unicos que aparecen en un anio, si el mismo codigo aparece en otro anio, este se mantiene.

In [ ]:
new_xls = []
lista_repetidos = []
lista_archivos = ['50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60']

for f in lista_archivos:
    if f in ['50', '51', '55', '56', '57']:
        xls_list = pd.read_excel(f"raw-data/eca/ECA {f}.xlsx").to_numpy().tolist()
    else:
        xls_list = pd.read_excel(f"raw-data/eca/ECA {f}.xlsx", sheet_name=['personas'])['personas'].to_numpy().tolist()
    for row in xls_list:
        if pd.isna( row[0] ):
            pass
        ##Cosas que se buscar
        #codigo, titulo, pi, inst_pi, nombre+apellido, inst, nac, genero y funcion
        if f in ['50', '52']:
            codigo, titulo, pi, inst_pi, nombre, apellido, inst, nac, genero, funcion = row
        elif f in ['51']:
            codigo, titulo, pi, inst_pi, nombre, apellido, inst, nac, genero = row
            funcion = 'noinf'
        elif f in ['53']:
            codigo, pi, nombre, apellido, _, genero, inst_pi, titulo = row
            inst = 'noinf'
            nac = 'noinf'
            funcion = 'noinf'
        elif f in ['57']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, _ = row
            inst_pi = 'noinf'
            funcion = 'noinf'
        elif f in ['54']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, _ , inst_pi, _= row
            funcion = 'noinf'
        elif f in ['55', '56']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, funcion, _ = row
            inst_pi = 'noinf'
        elif f in ['58', '59', '60']:
            codigo, titulo, pi, nombre, apellido, inst, nac, genero, funcion, _, inst_pi = row
        else:
            pass
        
        
        codigo = codigo.split( ' ' )[0].replace( '_', '-' )
        proyecto, tesis, _ = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        anio = 'noinf'
        if len( codigo.split('-') ) != 3:
            pass
        else:
            anio = f"20{ codigo.split('-')[2] }"

        titulo = limpiar_caracteres_especiales( titulo.lower() )

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        nombre = limpiar_caracteres_especiales( nombre.lower() ).replace('-', ' ').replace(',', '').replace('*', '').strip()
        apellido = limpiar_caracteres_especiales( apellido.lower() ).replace('-', ' ').replace(',', '').replace('*', '').strip()
        nombre = f"{ nombre } { apellido }".strip()
        nombre = normalizacion_nombres[nombre]


        inst = limpiar_caracteres_especiales( inst.lower().strip() )
        inst = normalizacion_universidades[ inst ]
        region_inst = region_institucion[inst]
        macro_inst = macrozona_region[region_inst]

        inst_pi = limpiar_caracteres_especiales( inst_pi.lower().strip() )
        inst_pi = normalizacion_universidades[ inst_pi ]
        region_inst_pi = region_institucion[inst_pi]
        macro_inst_pi = macrozona_region[region_inst_pi]

        nac = normalizacion_paises[ nac.strip().upper() ].lower().strip()

        funcion = limpiar_funciones( limpiar_caracteres_especiales( funcion.lower() ) )


        genero = genero.strip().upper()
        if genero == 'MASCULINO':
            genero = 'M'
        elif genero == 'FEMENINO':
            genero = 'F'
        elif genero == 'NOINF':
            genero = genero.lower()

        if [f, anio, codigo] not in lista_repetidos:
            new_xls.append([f, anio, codigo, instrumento, proyecto, inst_instrumento, tesis, titulo, 
                pi, inst_pi, region_inst_pi, macro_inst_pi])
            lista_repetidos.append( [f, anio, codigo] )
    print(f"{f} done")

In [7]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['eca', 'anio', 'codigo', 'instrumento', 'inst-instrumento', 'tipo-proyecto-inach', 'tesis', 
    'titulo', 'pi', 'inst-pi', 'region-inst-pi', 'macro-inst-pi'])
full_dataframe.to_excel(f"eca/ECA-just-codes-by-year.xlsx")

### Solo codigos
Solo se considera una aparicion del codigo y se almacena junto a su primera aparicion (numero de eca)

In [ ]:
new_xls = []
codigos_ya_considerados = []
lista_archivos = ['50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60']

for f in lista_archivos:
    if f in ['50', '51', '55', '56', '57']:
        xls_list = pd.read_excel(f"raw-data/eca/ECA {f}.xlsx").to_numpy().tolist()
    else:
        xls_list = pd.read_excel(f"raw-data/eca/ECA {f}.xlsx", sheet_name=['proyectos'])['proyectos'].to_numpy().tolist()
    for row in xls_list:
        if pd.isna( row[0] ):
            pass
        ##Cosas que se buscar
        #codigo, titulo, pi, inst_pi, ciudad
        if f in ['50', '52']:
            codigo, titulo, pi, inst_pi, _,_, _, _, _, _ = row
            ciudad = 'noinf'
        elif f in ['51']:
            codigo, titulo, pi, inst_pi, _, _, _, _, _ = row
            ciudad = 'noinf'
        elif f in ['53', '59', '60']:
            codigo, titulo, pi, inst_pi, _ = row
            ciudad = 'noinf'
        elif f in ['54', '58']:
            codigo, titulo, pi, inst_pi, ciudad, _ = row
        elif f in ['55', '56']:
            codigo, titulo, pi, _, _, _, _, _, _, _ = row
            ciudad = 'noinf'
        elif f in ['57']:
            codigo, titulo, pi, _, _, _, _, _, _ = row
            inst_pi = 'noinf'
            ciudad = 'noinf'

        codigo = codigo.split( ' ' )[0].replace( '_', '-' )
        anio = 'noinf'
        if len( codigo.split('-') ) != 3:
            pass
        else:
            anio = f"20{ codigo.split('-')[2] }"

        if codigo not in codigos_ya_considerados:
            proyecto, tesis, _ = descripcion_codigo_inach( codigo )
            instrumento = instrumento_financiamiento( codigo.split('-')[0] )
            inst_instrumento = institucion_instrumento( codigo )

            titulo = limpiar_caracteres_especiales( titulo.lower() )

            pi = limpiar_caracteres_especiales( pi.lower() ).strip()
            if '/' in pi:
                pi = pi.split('/')[0].strip()
            elif '-' in pi:
                pi = pi.split('-')[0].strip()
            pi = normalizacion_nombres[pi]
            
            inst_pi = limpiar_caracteres_especiales( inst_pi.lower().strip() ).strip()
            inst_pi = normalizacion_universidades[ inst_pi ]
            region_inst_pi = region_institucion[inst_pi]
            macro_inst_pi = macrozona_region[region_inst_pi]

            
            new_xls.append([f, anio, codigo, instrumento, proyecto, inst_instrumento, tesis, titulo, 
                pi, ciudad, inst_pi, region_inst_pi, macro_inst_pi, ])
            codigos_ya_considerados.append( codigo )
    print(f"{f} done")


In [10]:
full_dataframe = pd.DataFrame(np.array(new_xls), columns=['eca', 'anio-codigo', 'codigo', 'instrumento', 'inst-instrumento', 'tipo-proyecto-inach', 'tesis', 
    'titulo', 'pi', 'ciudad', 'inst-pi', 'region-inst-pi', 'macro-inst-pi'])
full_dataframe.to_excel(f"eca/ECA-just-codes.xlsx")